## Quadrature formulas

We're trying to find quadrature formulas on arbitrary triangles that satisfy

$$|T|\sum_i w_if(x_i) =  \int_T f(x)$$

and are exact for a certain polynomial degree $P_k$, $k>0$. Let $\{p_i\}_{1\le i\le n}$ be a basis for $P_k$. These can either be monomials of the form $x^iy^j$, but, for stability reasons, we instead choose orthogonal polynomials. The (nonlinear) system of equations we have to solve is given by

$$
A(x)w:=|T| \begin{pmatrix} 
p_1(x_1) & \ldots  & p_1(x_m) \\
\vdots &  &  \vdots \\
\vdots &  &  \vdots \\
p_n(x_1) & \ldots & p_n(x_m)
\end{pmatrix}
\begin{pmatrix} w_1 \\ \vdots \\ w_m \end{pmatrix}
=  \begin{pmatrix} \int_T p_1 \\ \vdots\\ \vdots \\ \int_T p_n \end{pmatrix}=: b
$$

Let's first think about how big these systems are. The number of rows is easy to determine, as corresponds to the desired polynomial degree $k$. In 2d, there are $n = 1/2(k+1)(k+2)$ basis functions spanning $P_k$. As for the number of columns... well this is more difficult. One can theoretically add as many points as one wants, but many of them may be redundant, meaning we could achieve the same degree of integration with fewer points. In general however, the optimal number of points required is less than the "size" of the polynomial degree. This means that $A(x)$ is a "tall" matrix, making the system of equations underdetermined.

However, the rank of $A(x)$ is still full, almost always. (why?)

## System to solve

Let us redefine the problem into a root-finding problem of the form

$$F(x,w) = A(x)w-b \stackrel{!}{=}0$$

Since the system is underdetermined, we can calculate 

$$w = w(x) = [A(x)^\top A(x)]^{-1}A(x)^\top b$$

and redefine the problem as

$$ F(x) = A(x)w(x)-b= A(x)[A(x)^\top A(x)]^{-1}A(x)^\top b-b \stackrel{!}{=} 0$$

depending solely on the parameter $x$. This would mean that the normal solution is exact and satisfies the original system. Note that $F:R^m\to R^n$, $m<n$. One way to solve this underdetermined system is by using Gauß-Newton.

## Jacobians

For this, we first need to determine the Jacobian of $F$, which we henceforth call $J_F$. For this,let us first define 

$$B(x) = A(x)^\top A(x),\qquad C(x) = B(x)^{-1}$$

Thus, differentiationg with respect to the $i$-th component of $x$, we get

$$\begin{align}
A_i'(x) &:= \partial_{x_i} A(x) \\
B_i'(x) &:= \partial_{x_i} B(x) = A_i'(x)^\top A(x) +  A(x)^\top A_i'(x) \\
C_i'(x) &:= \partial_{x_i} C(x) = -C(x)B_i'(x)C(x) \\
F_i'(x) &:= \partial_{x_i} F(x) = \partial_{x_i}[A(x)C(x)A(x)^\top b] \\
& = A_i'(x)C(x)A(x)^\top b -A(x)C_i'(x)A(x)^\top b + A(x)C(x)A_i'(x)^\top b
\end{align}
$$

Note: If one takes orthogonal polynomials, $b$ is zero up to the first entry. Also, the first polynomial is constant, and thus the first row in $A(x)$ does not depend on $x$. As such, $A_i'(x)^\top b=0$, and the third term vanishes, which yields

$$
F_i'(x) = A_i'(x)C(x)A(x)^\top b -A(x)C_i'(x)A(x)^\top b
$$

and $J_F(x) = \Big(F_1'(x) \;\ldots \; F_m'(x)\Big)$



## Gauss-Newton



For Gauss-Newton, we redefine the problem as a minimization problem of the form

$$\min_x \tfrac12 \| F(x)\|_2^2 $$

The first optimality condition is given by

$$G(x):=J_F(x)^\top F(x)\stackrel{!}{=} 0$$

with derivative (Jacobian)

$$ J_G(x) = J_F(x)^\top J_F(x) + "H_F(x) F(x)" \approx J_F(x)^\top J_F(x)$$

The higher order terms from the Hessian are ignored... making the whole thing an "inexact/quasi" Newton method. The Newton iteration reads

$$ x_{n+1} = x_n - J_G(x_n)^{-1}G(x_n) = x_n - [J_F(x_n)^\top J_F(x_n)]^{-1}J_F(x_n)^\top F(x_n)$$

This could also be derived as an exact Neton step for $F(x)=0$, where the inverse of the Jacobian of $F$ is computed using the pseudo-inverse. However, deriving it as the solution to an optimization problem has certain advantages, which we will touch upon in a bit.

## Barrier functions

This methods works fine... most of the time. However, it may happen that the points lie outside the triangle when the iteration converges... even if the initial guess only has points in the interior. Further, one might be only interested in quadrature rules that only use positive weights. The method presented above provides no guarantees that this will happen... To impose certain constarints, one might look into **barrier functions**, like a logarithmic one.

Let's say we want to impose $w(x)>0$, i.e. all weights are positive. Then the problem to solve turns into a constrained optimization problem, which reads

$$\min_x \tfrac12 \| F(x)\|_2^2 \qquad \text{subject to} \qquad w(x)>0$$

An equivalent problem is to instead solve a sequence of unconstrained optimization problems

$$\min_x \tfrac12 \| F(x)\|_2^2 -\tfrac1L\log(w(x)), \;\; L>0$$

by increasing the value of $L$ each time and starting the Gauss-Newton iteration with the last result.

The idea here being that if you start with a positive $w(x)$ and a moderate $L$, the minimizer is discouraged of driving the values close to zero and into the negative, where the logarithm is undefined. As the value of $L$ increases, the weights can get closer to zero, but still not negative. In the limit case, it is expected that we reach a solution for the original problem, but with positive weights. If $L$ is large enough, we can drop the logarithm term altogether, and solve the original problem, since the starting value is most likely close enough to not diverge to a solution with negative weights.